<a href="https://colab.research.google.com/github/dy018/project105/blob/main/105_ML_project_song.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ITI105 Machine Learning Project

Team members:
* Lye Suh Jeng (7487427Y)
* Lee Li Neng (6203055B)
* Lim Chan Boon (9704541M)
---
Project Problem: 4 (a) as in suggested project:
* The success of the song can often been measured by whether the song is on the Hit Chart such as Billboard Hot 100. It is important for music labels to know what makes a song successful so that they can focus their budget on making songs that has the highest chance of being successful.
---
We want to solve the problem statement by using the follow steps:
1.   Load dataset
2.   Discover & visualize data to gain insights
3.   Prepare data for Machine Learning
4.   Detect potential Multicollinearity and feature selections
5.   Split data into train and test datasets
6.   Train Base Models using the following algorithms:
      * Multiple Linear Regression (MLR)
      * Polynomial Regression (PR)
      * Elastic Net Regression (ENR)
      * Decision Tree Regressor (DTR)
7. Evaluate the performance of the base models using test data
8. Perform stacking on base model and evaluate performance
9. Deploy the model

# (1) Gather and Load dataset

In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('https://github.com/dy018/project105/blob/main/song_data.csv')
df.head()

https://github.com/dy018/project105.git

ParserError: Error tokenizing data. C error: Expected 1 fields in line 40, saw 19
